In [ ]:
from pathlib import Path
from freqtrade.configuration import Configuration

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

path='\\\\freqtrade\\user_data\\config\\'

config = Configuration.from_files([path + "config_binance_zone.json"])

data_location = Path(config['user_data_dir'], 'data', 'binance')

pair = "BTC_USDT"


In [ ]:
config["timeframe"] = '4h'

In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

dataFrame = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair)

# Confirm success
print("Loaded " + str(len(dataFrame)) + f" rows of data for {pair} from {data_location}")
dataFrame.head()

In [ ]:
dataFrame.head()

In [ ]:
dataFrame.sort_values(by='date',ascending = False)
dataFrame.info()

In [ ]:
dataFrame.plot('date',['close'])
plt.show()


In [ ]:
date_data = dataFrame.set_index('date', drop=False)
data_between = date_data['2018-01-01':'2019-01-01']
data_between

In [ ]:
data = dataFrame.set_index('date', drop=False)
data_red = data['2021-01-01':'2021-06-10']
data_red.plot('date',['close'])

In [ ]:
dataFrame

In [ ]:
### Strategy

In [ ]:
 dataFrame = dataFrame.set_index('date', drop=False)
 dataFrame = dataFrame.sort_index()
 dataFrame

In [ ]:
# Add your lib to import here
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
import math

emaFast = 12
emaSlow = 26
rsiPeriod = 14
overBought = 80
overSold = 30

# Fast Trail 
atrFast = 6
atrFM = 0.5 # fast ATR multiplier

# Slow Trail 
atrSlow = 18 # Slow ATR perod
atrSM = 2 # Slow ATR multiplier
# real = ATR(high, low, close, timeperiod=14)

dataFrame['ohlc4'] = (dataFrame['open'] + dataFrame['high'] + dataFrame['low'] + dataFrame['close']) / 4
dataFrame['ema_fast'] = ta.EMA(dataFrame, timeperiod=emaFast)
dataFrame['ema_slow'] = ta.EMA(dataFrame, timeperiod=emaSlow)
dataFrame['rsi'] = ta.RSI(dataFrame, timeperiod=rsiPeriod)
dataFrame['macd'] = dataFrame['ema_fast'] - dataFrame['ema_slow']
dataFrame['bullish'] = dataFrame['macd'] > 0
dataFrame['bearish'] = dataFrame['macd'] < 0

dataFrame.dropna(inplace=True)
dataFrame

In [ ]:

dataFrame['sl1'] = atrFM*ta.ATR(dataFrame.high, dataFrame.low, dataFrame.close,timeperiod=atrFast)  # Stop Loss
dataFrame['sl2'] = atrSM*ta.ATR(dataFrame.high, dataFrame.low, dataFrame.close,timeperiod=atrSlow) 

dataFrame.dropna(inplace=True)
# dataFrame.head()

In [ ]:
index = 0
dataFrame.iloc[index].close
dataFrame.iloc[index].date
dataFrame.iloc[index].close 
dataFrame_between01 = dataFrame['2019-06-26':'2019-06-27']
dataFrame_between01

In [ ]:
dataFrame['red'] = False
dataFrame['brown'] = False
dataFrame['yellow'] = False
dataFrame['blue'] = False
dataFrame['green'] = False
dataFrame['long'] = False
dataFrame['preBuy'] = False
dataFrame['short'] = False
dataFrame['preSell'] = False
dataFrame['trail2'] = 0.0

"""
Buy = bullish and bearish[1]
PreBuy = Blue and Blue[1] and Blue[2] and Blue[3] and mainSource<mainSource[2]
BuyMore = barssince(bullish)<26 and Yellow and mainSource==lowest(mainSource,9)
Sell = bearish and bullish[1]
PreSell = Yellow and barssince(Buy)>25 and mainSource<mainSource[2]
SellMore = Yellow and barssince(Yellow)>2 and mainSource<mainSource[2]
"""

for index in range(len(dataFrame)):
    dataFrame.green.iloc[index] = dataFrame.bullish.iloc[index] and (dataFrame.ohlc4.iloc[index] > dataFrame.ema_fast.iloc[index])
    dataFrame.blue.iloc[index] = dataFrame.bearish.iloc[index] and dataFrame.ohlc4.iloc[index] > dataFrame.ema_fast.iloc[index]
    dataFrame.yellow.iloc[index] = dataFrame.bullish.iloc[index] and dataFrame.ohlc4.iloc[index] < dataFrame.ema_slow.iloc[index]
    dataFrame.brown.iloc[index] = dataFrame.bullish.iloc[index] and dataFrame.ohlc4.iloc[index] < dataFrame.ema_fast.iloc[index] and dataFrame.ohlc4.iloc[index] < dataFrame.ema_slow.iloc[index]
    dataFrame.red.iloc[index] = dataFrame.bearish.iloc[index] and dataFrame.ohlc4.iloc[index] < dataFrame.ema_fast.iloc[index]
    if dataFrame.close.iloc[index] > dataFrame.trail2.iloc[index - 1] and dataFrame.close.iloc[index - 1] > dataFrame.trail2.iloc[index - 1]:
        dataFrame.trail2.iloc[index] = max(dataFrame.trail2.iloc[index - 1], dataFrame.close.iloc[index] - dataFrame.sl2.iloc[index])
    elif dataFrame.close.iloc[index] < dataFrame.trail2.iloc[index - 1] and dataFrame.close.iloc[index - 1] < dataFrame.trail2.iloc[index - 1]: 
        dataFrame.trail2.iloc[index] = min(dataFrame.trail2.iloc[index - 1], dataFrame.close.iloc[index - 1] +  dataFrame.sl2.iloc[index - 1])
    elif dataFrame.close.iloc[index] > dataFrame.trail2.iloc[index - 1]:
        dataFrame.trail2.iloc[index] = dataFrame.close.iloc[index] - dataFrame.sl2.iloc[index]
    else:
        dataFrame.trail2.iloc[index] = dataFrame.close.iloc[index] + dataFrame.sl2.iloc[index]
    # it can use rolling
    dataFrame.long.iloc[index] = dataFrame.bullish.iloc[index] and dataFrame.bullish.iloc[index - 1]
    dataFrame.preBuy.iloc[index] = dataFrame.bullish.iloc[index] and dataFrame.bullish.iloc[index - 1]
    dataFrame.short.iloc[index] = dataFrame.bearish.iloc[index] and dataFrame.bearish.iloc[index - 1]

dataFrame['greenLine'] = False
dataFrame.loc[
        (
            (dataFrame["close"] > dataFrame['trail2'])
        ),
        'greenLine'] = True
dataFrame['greenLine_last'] = dataFrame.greenLine.shift(1)

dataFrame['short_last'] = dataFrame.short.shift(1)
dataFrame['green_last'] = dataFrame.green.shift(1)
dataFrame['red_last'] = dataFrame.red.shift(1)
dataFrame['hold_state'] = False
dataFrame.dropna(inplace=True)
dataFrame

In [ ]:

dataFrame['hold_state'].shift(1) == False & (dataFrame['greenLine_last'] == False | (dataFrame['greenLine_last'] & dataFrame['greenLine'])) & (dataFrame['long'] | dataFrame['green'] & dataFrame['greenLine']) | (dataFrame['blue'] & dataFrame['greenLine'])

In [ ]:
dataFrame['signal_buy'] = False
dataFrame['signal_sell'] = False
dataFrame['on_buy'] = 0

dataFrame.loc[(
    (dataFrame['green_last'] == False) & (dataFrame['green'] == True) # Green buy
    | ((dataFrame['greenLine'] == True) & (dataFrame['blue'] == True)) # Over ATR and blue
), 'signal_buy'] = True

dataFrame.loc[(
    ((dataFrame['red_last'] == False) & (dataFrame['red'] == True)) # Red Sell
    | ((dataFrame['greenLine_last'] == True) & (dataFrame['greenLine'] == False)) # Stop lost
), 'signal_sell'] = True

dataFrame.loc[(
   (dataFrame['signal_buy'] == True)
), 'on_buy'] = 1



In [ ]:
dataFrame.loc[(dataFrame['signal_buy'] == True) , 'buy'] = 1
dataFrame.loc[(dataFrame['signal_sell'] == True), 'sell'] = 1

In [ ]:
dataFrame_between = dataFrame['2020-01-26':'2020-01-30']
dataFrame_between.loc[:,['ohlc4','ema_fast','macd','bullish','bearish','green','signal_buy','signal_sell','blue','yellow','brown','red_last','red','greenLine_last','greenLine','buy','on_buy','sell']]

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
graph = generate_candlestick_graph(pair=pair,
                                   data=dataFrame_between,
                                   #trades=dataFrame,
                                   indicators1=['ema_fast', 'ema_slow', 'trail2','greenLine','on_buy','buy','sell'],
                                   indicators2=['rsi']
                                  )
graph.show()

In [ ]:
dataFrame_between01 = dataFrame['2019-06-26':'2019-06-27']
dataFrame_between01.loc[:,['ohlc4','ema_fast','macd','bullish','bearish','green','blue','yellow','brown','red','sl1','sl2','long','preBuy','short','preSell','trail2','greenLine','greenLine_last','short_last','buy','sell']]

In [ ]:
a = dataFrame_between01.iloc[0]
a.bullish and a.ohlc4 > a.ema_fast

In [ ]:
dataFrame_between = dataFrame['2019-08-06':'2019-08-11']
dataFrame_between.loc[:,['ohlc4','bearish','ema_fast','green','blue','yellow','brown','red','preSell','greenLine','greenLine_last','short_last','buy','sell']]

In [ ]:
graph = generate_candlestick_graph(pair=pair,
                                   data=dataFrame_between,
                                   indicators1=['ema_fast', 'ema_slow', 'trail2','greenLine'],
                                   indicators2=['rsi']
                                  )
graph.show()

In [ ]:

#trades_red = trades.loc[trades['pair'] == pair]
graph = generate_candlestick_graph(pair=pair,
                                   data=dataFrame,
                                   indicators1=['ema_fast', 'ema_slow', 'trail2'],
                                   indicators2=['rsi']
                                  )
graph.show(renderer="browser")